In [37]:
import timm

import torch
import torch.nn as nn
from torchvision import transforms, models


class MyModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.base_model = models.efficientnet_b7(pretrained=True)
        self.feature_extractor = torch.nn.Sequential(
            *list(self.base_model.children())[:-2],
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
        self.avg_pooler = nn.AdaptiveAvgPool1d(64)

    def forward(self, x):
        x = transforms.functional.resize(x,size=[256, 256])
        x = transforms.functional.center_crop(x, 224)

        x = x/255.0
        x = transforms.functional.normalize(x, 
                                                mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])
        x = self.feature_extractor(x)
        x = self.avg_pooler(x)
        
        return x


model = MyModel()
model.eval()
pass

In [38]:
# model

In [39]:
# print(model.base_model(torch.rand(2, 3, 500,500)).shape)
print(model(torch.rand(2, 3, 500,500)).shape)

torch.Size([2, 64])


In [ ]:
saved_model = torch.jit.script(model)
saved_model.save('../input/google-img-embed/v6_efficientnet_b7.pt')

In [18]:
from torchvision.models import efficientnet_b7

In [20]:
model = efficientnet_b7(pretrained=True)

In [27]:
torch.nn.Sequential(
    *list(model.children())[:-2],
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.AdaptiveAvgPool1d(64)
    )(torch.rand(2,3,244,244)).shape

torch.Size([2, 64])

In [13]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (bn1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    